In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
el_pais_20240404 = pd.read_csv('raw_data/el_pais_20240404.csv')
el_pais_20240413 = pd.read_csv('raw_data/el_pais_20240413.csv')
el_pais_20240509 = pd.read_csv('raw_data/el_pais_20240509.csv')
el_mundo_20240404 = pd.read_csv('raw_data/el_mundo_20240404.csv')
el_mundo_20240413 = pd.read_csv('raw_data/el_mundo_20240413.csv')
el_mundo_20240509 = pd.read_csv('raw_data/el_mundo_20240509.csv')

In [3]:
periodico_el_pais = 'El País'
periodico_el_mundo = 'El Mundo'
el_pais_20240404 = el_pais_20240404.assign(periodico=periodico_el_pais, fecha_scrap = '2024-04-04')
el_pais_20240413 = el_pais_20240413.assign(periodico=periodico_el_pais, fecha_scrap = '2024-04-13')
el_pais_20240509 = el_pais_20240509.assign(periodico=periodico_el_pais, fecha_scrap = '2024-05-09')
el_mundo_20240404 = el_mundo_20240404.assign(periodico=periodico_el_mundo, fecha_scrap = '2024-04-04')
el_mundo_20240413 = el_mundo_20240413.assign(periodico=periodico_el_mundo, fecha_scrap = '2024-04-13')
el_mundo_20240509 = el_mundo_20240509.assign(periodico=periodico_el_mundo, fecha_scrap = '2024-05-09')

In [4]:
total_df_raw = resultado = pd.concat([el_pais_20240404, el_pais_20240413,el_pais_20240509,
                                      el_mundo_20240404, el_mundo_20240413, el_mundo_20240509], ignore_index=True)

In [5]:
total_df_raw.head()

,categoria,enlace,titulo,texto,fecha,autor,periodico,fecha_scrap
0,Opinión,https://elpais.com/opinion/2024-04-04/la-digni...,La dignidad de Mónica Oltra,Mónica Oltra ha sido exonerada en la causa jud...,2024-04-04T05:00:00+02:00,El País,El País,2024-04-04
1,Opinión,https://elpais.com/opinion/2024-04-04/giro-en-...,Giro en Turquía,Por primera vez desde que fuera fundado hace 2...,2024-04-04T05:00:00+02:00,El País,El País,2024-04-04
2,Opinión,https://elpais.com/opinion/2024-04-04/la-perve...,La perversión del lenguaje o la banalización d...,No descubro la sopa de ajo si digo que estamos...,2024-04-04T05:00:00+02:00,Carmen Domingo,El País,2024-04-04
3,Opinión,https://elpais.com/opinion/2024-04-04/revoluci...,Revolución no es lo mismo que transición,Con el cincuentenario de\n la revolución portu...,2024-04-04T05:00:00+02:00,NaN,El País,2024-04-04
4,Opinión,https://elpais.com/opinion/2024-04-01/un-cuerv...,‘Un cuervo come’,NaN,2024-04-01T05:00:00+02:00,NaN,El País,2024-04-04


In [6]:
total_df_raw.isna().sum()

categoria        0
enlace           0
titulo           0
texto           57
fecha           25
autor          153
periodico        0
fecha_scrap      0
dtype: int64

In [7]:
total_df = total_df_raw.dropna(subset=['texto'])
total_df = total_df.drop_duplicates(subset=['enlace'])

In [8]:
total_df.shape

(1446, 8)

In [9]:
total_df.periodico.value_counts(dropna=False)

periodico
El Mundo    985
El País     461
Name: count, dtype: int64

In [10]:
total_df['longitud_texto'] = total_df['texto'].apply(len)

In [11]:
grouped_by_categoria = total_df.groupby("periodico")["categoria"].value_counts().unstack(fill_value=0)
grouped_by_categoria

categoria,Ciencia,Cultura,Deportes,Economía,España,Opinión
periodico,,,,,,
El Mundo,90,177,180,179,193,166
El País,82,99,87,82,55,56


In [12]:
grouped_by_len = total_df.groupby('periodico')['longitud_texto'].agg(['min', 'max', 'mean'])
grouped_by_len

,min,max,mean
periodico,,,
El Mundo,45,21820,3151.137056
El País,71,21471,5589.659436


In [13]:
muestras = total_df.groupby(['periodico', 'categoria']).apply(lambda x: x.sample(n=50, random_state=42))
muestras = muestras.reset_index(drop=True)

In [14]:
grouped_by_categoria = muestras.groupby("periodico")["categoria"].value_counts().unstack(fill_value=0)
grouped_by_categoria

categoria,Ciencia,Cultura,Deportes,Economía,España,Opinión
periodico,,,,,,
El Mundo,50,50,50,50,50,50
El País,50,50,50,50,50,50


In [15]:
grouped_by_len = muestras.groupby('periodico')['longitud_texto'].agg(['min', 'max', 'mean'])
grouped_by_len

,min,max,mean
periodico,,,
El Mundo,47,12463,3332.940000
El País,71,17270,5507.343333


In [16]:
muestras.head()

,categoria,enlace,titulo,texto,fecha,autor,periodico,fecha_scrap,longitud_texto
0,Ciencia,https://www.elmundo.es/ciencia-y-salud/ciencia...,Defensa planetaria: los científicos creen que ...,La primera operación espacial para desviar vol...,2024-02-28T10:51:21Z,Teresa Guerrero,El Mundo,2024-04-04,5045
1,Ciencia,https://www.elmundo.es/ciencia-y-salud/ciencia...,Captan una bola de fuego sobrevolando Portugal...,"Los detectores del proyecto \nSMART\n, del Ins...",2024-03-16T19:51:02Z,Efe Huelva,El Mundo,2024-04-04,2132
2,Ciencia,https://www.elmundo.es/ciencia-y-salud/ciencia...,El cometa 'diablo' ya es visible en el cielo n...,"El cometa\n '12P/Pons-Brooks'\n, conocido popu...",2024-04-08T09:41:45Z,Efe Madrid,El Mundo,2024-04-13,3174
3,Ciencia,https://www.elmundo.es/ciencia-y-salud/ciencia...,Pere Estupinyà y Lluís Montoliu ganan el premi...,Pere Estupinyà y Lluís Montoliu acaban de ser ...,2024-04-30T11:52:48Z,EL MUNDO Madrid,El Mundo,2024-05-09,4849
4,Ciencia,https://www.elmundo.es/ciencia-y-salud/ciencia...,Un estudio relaciona ser zurdo con variantes r...,Ser zurdo \no diestro depende del hemisferio c...,2024-04-04T13:18:21Z,Efe,El Mundo,2024-04-04,1753


# Preprocesamiento

In [17]:
def cargar_stopwords(archivo_stopwords):
    with open(archivo_stopwords, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return stopwords

def limpiar_texto(texto):
    # eliminar intros y otros caracteres no deseados
    texto_limpio = re.sub(r'\n', ' ', texto)  # eliminar intros
    texto_limpio = re.sub(r'[^\w\s]', '', texto_limpio)  # eliminar caracteres especiales excepto puntos
    # cambiar tildes por la misma letra sin tilde
    # texto_limpio = unidecode(texto_limpio)
    # convertir texto a minúsculas
    texto_limpio = texto_limpio.lower()
    # tokenizar el texto en palabras
    palabras = texto_limpio.split()
    # eliminar stopwords
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_espanol]
    # reconstruir el texto sin las stopwords
    texto_sin_stopwords = ' '.join(palabras_filtradas)
    return texto_sin_stopwords

# cargar stopwords en español
stopwords_espanol = cargar_stopwords('spanish_stop_words.txt')

In [18]:
muestras['texto_limpio'] = muestras['texto'].apply(limpiar_texto)
muestras['titulo_limpio'] = muestras['titulo'].apply(limpiar_texto)

In [19]:
print(muestras.texto[0])

La primera operación espacial para desviar voluntariamente la trayectoria de un asteroide y ensayar una futura misión de defensa planetaria sigue siendo analizada por los científicos, que estudian hasta qué punto el 
impacto de la sonda 
de la NASA DART contra el asteroide Dimorphos 
alteró
 la órbita y composición de esta roca espacial de 160 metros de diámetro el 26 de septiembre de 2022, cuando se produjo ese choque controlado.
El asteroide Dimorphos 
orbita otro asteroide más grande, denominado Didymos, 
de 700 metros de diámetro, conformando lo que se conoce como un sistema binario. De ahí el nombre de la misión, DART, que es el acrónimo de 
Double Asteroid Redirection Test
 pero también significa 
dardo 
en inglés. 
Una nueva investigación publicada esta semana en la revista 
Nature Astronomy 
ofrece nuevos datos tras realizar una simulación de esta misión estadounidense. En primer lugar, los resultados del equipo suizo liderado por Sabina Raducan, de la Universidad de Berna, apu

In [20]:
print(muestras.texto_limpio[0])

primera operación espacial desviar voluntariamente trayectoria asteroide ensayar futura misión defensa planetaria sigue siendo analizada científicos estudian punto impacto sonda nasa dart asteroide dimorphos alteró órbita composición roca espacial 160 metros diámetro 26 septiembre 2022 produjo choque controlado asteroide dimorphos orbita asteroide grande denominado didymos 700 metros diámetro conformando conoce sistema binario ahí nombre misión dart acrónimo double asteroid redirection test significa dardo inglés nueva investigación publicada semana revista nature astronomy ofrece nuevos datos tras realizar simulación misión estadounidense primer lugar resultados equipo suizo liderado sabina raducan universidad berna apuntan choque dejado cráter permanente cuerpo espacial pensaba sino remodelado completo asteroide así creen dimorphos puede haber cambiado forma según afirman indicaría trata débil pila escombros formada material desprendido asteroide didymos saber si razón esperar inspec

In [21]:
print(muestras.titulo[0])

Defensa planetaria: los científicos creen que el impacto de la sonda DART contra el asteroide Dimorphos lo remodeló por completo


In [22]:
print(muestras.titulo_limpio[0])

defensa planetaria científicos creen impacto sonda dart asteroide dimorphos remodeló completo


In [23]:
muestras.to_csv('new_processed.csv', index=False)

# Test subset

In [24]:
def sample_group(group):
    return group.sample(n=6, replace=True) 

submuestra = muestras.groupby(['periodico', 'categoria']).apply(sample_group).reset_index(drop=True)
print(submuestra)

   categoria                                             enlace  \
0    Ciencia  https://www.elmundo.es/ciencia-y-salud/ciencia...   
1    Ciencia  https://www.elmundo.es/ciencia-y-salud/ciencia...   
2    Ciencia  https://www.elmundo.es/ciencia-y-salud/ciencia...   
3    Ciencia  https://www.elmundo.es/ciencia-y-salud/ciencia...   
4    Ciencia  https://www.elmundo.es/ciencia-y-salud/ciencia...   
5    Ciencia  https://www.elmundo.es/ciencia-y-salud/ciencia...   
6    Cultura  https://www.elmundo.es/cultura/musica/2024/05/...   
7    Cultura  https://www.elmundo.es/cultura/toros/2024/05/0...   
8    Cultura  https://www.elmundo.es/cultura/musica/2024/03/...   
9    Cultura  https://www.elmundo.es/uestudio/2024/03/27/660...   
10   Cultura  https://www.elmundo.es/cultura/2024/04/03/660d...   
11   Cultura  https://www.elmundo.es/cultura/musica/2024/04/...   
12  Deportes  https://www.elmundo.es/deportes/tenis/2024/05/...   
13  Deportes  https://www.elmundo.es/deportes/formula-1/2024..

In [25]:
submuestra.to_csv('news_test.csv', index=False)